In [70]:
import os 
import pydicom
import numpy as np 
import matplotlib.pyplot as plt
import ipyvolume as ipv
import volumentations as V
import nibabel as nib

In [71]:
dicom_folder  = r"C:\KAGGLE\MEDICINE\data\train_images\10004\51033"

In [72]:
lst_files = os.listdir(dicom_folder)

In [4]:
def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    """
    Source : https://www.kaggle.com/competitions/rsna-2023-abdominal-trauma-detection/discussion/427217
    """
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
#         pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dcm)

    intercept = float(dcm.RescaleIntercept)
    slope = float(dcm.RescaleSlope)
    center = int(dcm.WindowCenter)
    width = int(dcm.WindowWidth)
    low = center - width / 2
    high = center + width / 2    
    
    pixel_array = (pixel_array * slope) + intercept
    pixel_array = np.clip(pixel_array, low, high)

    return pixel_array

In [25]:
def process(data_path="", size=512):
    lst_files = os.listdir(data_path)
    lst_files = [int(x[:-4]) for x  in lst_files]
    print(len(lst_files))
    lst_files.sort()
    if len(lst_files) > 1200:
        lst_files = lst_files[150:-150]
    elif len(lst_files) > 900:
        lst_files = lst_files[100:-100]
    print(len(lst_files))
    
    imgs = []
    if len(lst_files) > 1500:
        step = 5
    elif len(lst_files) > 1200:
        step = 3
    elif
    elif len(lst_files) > 600:
        step = 2
    else:
        step = 1
    step = 1
    for f in range(min(lst_files), max(lst_files) + 1, step):
        path_to_files = os.path.join(data_path, f"{f}.dcm")

        dicom = pydicom.dcmread(path_to_files)

        pos_z = dicom[(0x20, 0x32)].value[-1]

        img = standardize_pixel_array(dicom)
        img = (img - img.min()) / (img.max() - img.min() + 1e-6)

        if dicom.PhotometricInterpretation == "MONOCHROME1":
            img = 1 - img

    
        if img.shape != (512, 512):
            img = cv2.resize(img, (size, size))

        imgs.append(img)

    combined_array = np.stack(imgs, axis=0)
    
    combined_array = np.transpose(combined_array, [1, 2, 0])
    return combined_array


In [ ]:
img_new = process(data_path=dicom_folder, size=512)
img = img_new
img.shape

In [64]:
def create_3D_segmentations(filepath, downsample_rate=1):
    """
    Стандартные лейблы были:
    1 - liver 
    2 - spleen
    3 - kidney_left
    4 - kidney_right
    5 - bowel 
    Изменю структуру, чтобы было более удобно на такой вариант 

    1 - liver 
    2 - spleen 
    3 - kidney (r & l)
    4 - bowel 
    """
    img = nib.load(filepath).get_fdata()
    img = np.transpose(img, [1, 0, 2])
    img = np.rot90(img, 1, (1,2))
    img = img[::-1,:,:]
    img = np.transpose(img, [1, 0, 2])
    img = img[::downsample_rate, ::downsample_rate, ::downsample_rate]
    img = np.transpose(img, [1, 2, 0])
    img = np.round(img).astype(int)

    # 4 -> 3 
    img = np.where(img == 4, 3, img)
    # 5 -> 4
    img = np.where(img == 5, 4, img)
    print(img.shape)
    print(img.shape[2])
    print('aaaaa')
    if img.shape[2] > 1300:
        img = img[...,150:-150]
    elif img.shape[2] > 1000:
        img = img[...,100:-100]
    print(img.shape)
    

    if img.shape[2] > 1500:
        step = 5
    elif img.shape[2] > 1200:
        step = 3
    elif img.shape[2] > 800:
        step = 2
    else:
        step = 1
    step = 1
    return img[..., ::step]

In [75]:
import pandas as pd
data = pd.read_csv("../src_upd/data_for_segmentation.csv")
data.sort_values("count_layer").tail(20)

,patient_id,series_id,count_layer,size,count_layers_seg,same
29,15876,38633,725,"(512, 512, 725)",725,True
172,55888,13925,730,"(512, 512, 730)",730,True
3,10228,30522,747,"(512, 512, 747)",747,True
202,7642,778,751,"(512, 512, 751)",751,True
159,52416,54830,762,"(512, 512, 762)",762,True
112,38238,32670,763,"(512, 512, 763)",763,True
12,11652,39013,768,"(512, 512, 768)",768,True
145,47263,57769,768,"(512, 512, 768)",768,True
139,45488,30064,773,"(512, 512, 773)",773,True
155,50873,18207,788,"(512, 512, 788)",788,True


In [68]:
%%time
# ТОЛЬКО ПОЧКИ 
path_to_mask = r"C:\KAGGLE\MEDICINE\data\segmentations\21282.nii"
mask_data = create_3D_segmentations(path_to_mask)
print(mask_data.shape)


(512, 512, 850)
850
aaaaa
(512, 512, 850)
(512, 512, 850)
CPU times: total: 2.97 s
Wall time: 4.32 s


In [69]:
# Создайте трехмерную визуализацию
fig = ipv.figure()

# Отобразите объемные данные
# vol = ipv.volshow( img +  mask_data, lighting=True)
vol = ipv.volshow(   mask_data )
# Добавьте интерактивность (вращение, масштабирование и т.д.)
ipv.style.box_off()
ipv.style.axes_off()
ipv.style.set_style_light()

# Покажите визуализацию
ipv.show()

Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

In [21]:
import torch.nn.functional as F
import torch

In [22]:
img_tensor =  F.interpolate(
            torch.tensor(img).unsqueeze(0).unsqueeze(0),
            size=(192, 192, 192),
            mode='trilinear',
            align_corners=False
        )[0][0]

In [106]:
img_tensor.shape

torch.Size([192, 192, 192])

(512, 512, 522)


In [134]:
segmentaion_aug = V.Compose([
    V.Resize((192, 192, 192), interpolation=3, resize_type=0, always_apply=True, p=1.0),
], p=1.0)
data_for_aug = {"image": img, 'mask': mask_data}
new_data = segmentaion_aug(**data_for_aug)

In [135]:
upd_mask = new_data['mask']

In [136]:
np.unique(upd_mask)

array([0., 1., 2., 3., 4.], dtype=float32)

In [120]:
upd_img = new_data['image']

In [111]:
img_tensor = img_tensor.to('cpu').numpy()

In [137]:
mask_data = np.round(upd_mask).astype(int)
condition = ((mask_data != 2) )

mask_data = np.where(condition, 0, 1) 
np.unique(mask_data), mask_data.shape

(array([0, 1]), (192, 192, 192))

In [113]:
# img_new = img[:,:,400:800]
# img_new.shape

E:\Project_python\kaggle_3d\myenv\lib\site-packages\ipyvolume\serialize.py:102: RuntimeWarning: invalid value encountered in cast
  subdata[..., i] = ((gradient[i][zindex] / 2.0 + 0.5) * 255).astype(np.uint8)


Container(children=[VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step…

In [ ]:
mask_data.shape

In [ ]:
img.shape

In [ ]:

mask_data = nib.load(path_to_mask).get_fdata()
mask_data = np.round(mask_data).astype(int)

condition = ((mask_data != 3) & (mask_data != 4)) 
mask_data = np.where(condition, 0, 1) 
np.unique(mask_data)

In [ ]:
mask_data.shape

In [ ]:
def create_3D_segmentations(filepath, downsample_rate=1):
    img = nib.load(filepath).get_fdata()
    img = np.transpose(img, [1, 0, 2])
    img = np.rot90(img, 1, (1,2))
    img = img[::-1,:,:]
    img = np.transpose(img, [1, 0, 2])
    img = img[::downsample_rate, ::downsample_rate, ::downsample_rate]
    return img

In [ ]:
mask_data = create_3D_segmentations("../data/segmentations/7334.nii")
mask_data = np.round(mask_data).astype(int)

In [ ]:
mask_data.shape